# Using your our own glacier inventory with OGGM

[The Randolph Glacier Inventory](https://www.glims.org/RGI/) is a key dataset to model glaciers at any scale: it includes outlines of the extent of each glacier in the world, an information that is critical in figuring out how much a particular glacier might contribute to rising sea level. These glacier outlines are the starting point of any simulation in OGGM. The RGI's latest version (v6), as well as v5, are provided and supported by OGGM (see the [documentation](https://oggm.readthedocs.io/en/latest/input-data.html#glacier-outlines-and-intersects)). However, there are [several issues](https://rgitools.readthedocs.io/en/latest/known-issues.html) in the RGI that might make you want to use your own corrected glacier outlines. 

This notebook describes how to feed OGGM with them. We will show you three case studies about how to give any geometry to OGGM and avoid errors of incompatibility between your shapefile and the model framework.

We have three case studies which should cover a number of applications:
1. [Dividing a glacier into smaller entities](#case1) (common case, useful for poorly outlined glaciers, which are in reality separate dynamical entities)
2. [Merging two glaciers together](#case2) (useful for tidewater glaciers in particular, not much elsewhere)
3. [Start from a completely independent inventory](#case3)

## TLDR;

If you want to use custom data to feed OGGM with, you should:
- **make a shapefile that resembles the RGI one: same attributes, and the glacier geometries should be in lon/lat projection**. The most important attribute is `geometry`, of course, but others are used by OGGM as well: refer to [the OGGM documentation](https://docs.oggm.org/en/stable/input-data.html#glacier-outlines-and-intersects) to decide which ones. The RGI documentation (found in the RGI directory after download) is useful as well!
- **compute and use a new [glacier interesects](https://rgitools.readthedocs.io/en/latest/tools.html#glacier-intersects) file**, or make sure you don't need one and disable this option in OGGM. 

## Structure of an RGI file

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
import os
from oggm import cfg, utils, workflow, tasks, graphics
from oggm.core import inversion
cfg.initialize(logging_level='WARNING')

Let's read a file from the standard RGI:

In [ ]:
utils.get_rgi_dir(version='61')

In [ ]:
sh = utils.get_rgi_region_file('11', version='61')
sh

Shapefiles are best read an manipulated with [geopandas](http://geopandas.org/) in python (see also our [working_with_rgi.ipynb](working_with_rgi.ipynb) tutorial):

In [ ]:
gdf = gpd.read_file(sh)
gdf.head()

An RGI file contains the actual glacier geometries, but also a number of attribute which are used by OGGM afterwards. Let's learn how to make our own file now. 

<a id='case1'></a>
## Case 1: dividing a glacier into smaller entities

A typical example of wrongly divided glacier is Hintereisferner, in Austria:

In [ ]:
# OGGM set-up
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='rgi-case-1', reset=True)
cfg.PARAMS['border'] = 10

# Get the HEF geometry and plot it
gl = utils.get_rgi_glacier_entities(['RGI60-11.00897'])
gl.plot(edgecolor='k');

Obviously, the two smaller tongues used to flow in the main one but this is not the case anymore today. We need updated geometries.

### Make a new "RGI file"

There is no simple way to automate the process of finding these bad geometries, but we are [working on this](https://github.com/OGGM/partitioning) (don't hold your breath, this has been in development since a long time). Here we use a geometry that we prepared in QGis:

In [ ]:
# We simulate the case where we only have the geometry, nothing else
divides = gpd.read_file(utils.get_demo_file('divides_alps.shp'))
divides = divides.loc[divides.RGIId == 'RGI50-11.00897'][['geometry']]
divides

In [ ]:
divides.plot(edgecolor='k');

Now we use the RGI entity as template - it's good to use the same attributes as the original RGI glacier, because most of them are already correct:

In [ ]:
template = pd.concat([gl]*3, ignore_index=True)
template

We change the important ones:

In [ ]:
# Attributes
template['RGIId'] = ['RGI60-11.00897_d01', 'RGI60-11.00897_d02', 'RGI60-11.00897_d03']
template['Name'] = ['Hintereisferner d01', 'Hintereisferner d02', 'Hintereisferner d03']
# Geometries
template['geometry'] = divides['geometry'].values
# Center point
for i, geom in template[['geometry']].iterrows():
    cenlon, cenlat = geom.geometry.centroid.xy
    template.loc[i, 'CenLon'] = cenlon
    template.loc[i, 'CenLat'] = cenlat
# This is important to properly georeference the file
import salem
template.crs = salem.wgs84.srs

In [ ]:
template.plot(edgecolor='k');

Save it: 

In [ ]:
hef_new_shape_path = os.path.join(cfg.PATHS['working_dir'], 'hef_divided.shp')
template.to_file(hef_new_shape_path)

### Compute the intersects

Hintereisferner has a divide with another glacier as well! Let's find out which:

In [ ]:
intersects_alps = gpd.read_file(utils.get_rgi_intersects_region_file('11'))
intersects_hef = intersects_alps.loc[(intersects_alps.RGIId_1 == 'RGI60-11.00897') | (intersects_alps.RGIId_2 == 'RGI60-11.00897')]
intersects_hef

Ok, we can now create a file which has all the glaciers we need to compute the relevant intersects (note that we could also use the full standard RGI with just HEF replaced):

In [ ]:
for_intersects = pd.concat([template, utils.get_rgi_glacier_entities(['RGI60-11.00846'])], ignore_index=True)
for_intersects.crs = salem.wgs84.srs
for_intersects.plot(edgecolor='k');

Good! Let's use [rgitools](https://rgitools.readthedocs.io) to compute the intersects for this new situation: 

In [ ]:
from rgitools.funcs import compute_intersects
new_intersects = compute_intersects(for_intersects)

If this raises an error, you might have to install [rgitools](https://rgitools.readthedocs.io) first (e.g. `pip install git+https://github.com/OGGM/rgitools.git`)
and also [networkx](https://networkx.org/documentation/stable/index.html) (`pip install networkx`)

In [ ]:
f, ax = plt.subplots()
for_intersects.plot(ax=ax, edgecolor='k');
new_intersects.plot(ax=ax, edgecolor='r');

Good! We can store our intersects to use them with OGGM afterwards:

In [ ]:
hef_intersects_path = os.path.join(cfg.PATHS['working_dir'], 'hef_divided_intersects.shp')
new_intersects.to_file(hef_intersects_path)

### Finally: the OGGM run 

In [ ]:
# This is important! We tell OGGM to recompute the glacier area for us
cfg.PARAMS['use_rgi_area'] = False
# This is the default anyway, but we set it here to be sure
cfg.PARAMS['use_intersects'] = True

# This is important!
cfg.set_intersects_db(hef_intersects_path)

# This is to avoid a download in the tutorial, you dont' need do this at home
cfg.PATHS['dem_file'] = utils.get_demo_file('hef_srtm.tif')

# This is important again - standard OGGM 
rgidf = gpd.read_file(hef_new_shape_path)
gdirs = workflow.init_glacier_directories(rgidf, reset=True, force=True)

In [ ]:
workflow.execute_entity_task(tasks.define_glacier_region, gdirs);
workflow.execute_entity_task(tasks.glacier_masks, gdirs);
workflow.execute_entity_task(tasks.compute_centerlines, gdirs);
workflow.execute_entity_task(tasks.initialize_flowlines, gdirs);
workflow.execute_entity_task(tasks.catchment_area, gdirs);
workflow.execute_entity_task(tasks.catchment_width_geom, gdirs);
workflow.execute_entity_task(tasks.catchment_width_correction, gdirs);

If you get "NameError 'skdraw' is not defined", you have to install the python image processing package [scikit-image](https://scikit-image.org/docs/stable/install.html#installation-via-pip-and-conda) first (e.g. via `conda install scikit-image`)

In [ ]:
graphics.plot_catchment_width(gdirs, add_intersects=True, corrected=True)

It works!

**The intersects in OGGM are used for two main things:**
- when a grid-point glacier section touches an intersect, it will be attributed a rectangular bed (instead of a parabolic one)
- when interpolating the ice thickness to a 2D grid, the boundary condition thickness=0 at the glacier outline is removed where there are intersects

**We recommend to use intersects for your runs as well.**

<a id='case2'></a>
## Case 2: merging glaciers 

Sometimes, you may want to *merge* glaciers together. This case is less frequent than Case 1, but might be useful for calving glaciers, which are sometimes divided in the RGI.

### Original RGI outlines

We use a case study for two marine-terminating glaciers in Alaska that have to be merged into a single outline in order to model a correct calving flux for these glaciers (following the methods described in
[Recinos et al., (2019)](https://www.the-cryosphere.net/13/2657/2019/tc-13-2657-2019.html)). The resulting shapefile is a new one that needs to be adapted in order for OGGM to run.

We will study the Sawyer Glacier (`RGI60-01.03890`) that is actually connected via the calving front with this other entity (`RGI60-01.23664`). Visit this [link](https://glacierchange.wordpress.com/2011/03/16/saywer-glacier-alaska-retreat/) to learn more about the retreat of the Sawyer Glacier and see images illustrating this connection.

In [ ]:
gl = utils.get_rgi_glacier_entities(['RGI60-01.03890', 'RGI60-01.23664'])

Here OGGM downloaded the outlines for both glaciers. If we plot them together, we can see that both glaciers drain into the same fjord. See the google map below:

In [ ]:
cfg.initialize(logging_level='WARNING')
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='rgi-case-2-example', reset=True)
cfg.PARAMS['border'] = 10
gdirs = workflow.init_glacier_directories(['RGI60-01.03890', 'RGI60-01.23664'], from_prepro_level=3, reset=True, force=True)
graphics.plot_googlemap(gdirs, figsize=(6, 6));

The upper glacier map is a zoom version of the plot below. 
They share the same glaciers terminus. Therefore, to estimate a calving flux for these glaciers we need them connected. 

### Let's merge these two outlines using geopandas

In [ ]:
merged = gl.dissolve(by='O2Region', as_index=False) 
merged = merged[gl.columns]
merged.plot(edgecolor='k');

### RGI attributes 

We now have a new shapefile, which resembles an RGI one but has wrong attributes. Some aren't relevant, but some are. See the [documentation](https://oggm.readthedocs.io/en/latest/input-data.html#glacier-outlines-and-intersects) for a list. 

The important ones are: RGIId, CenLon, CenLat, TermType, Area. Area and CenLon, Cenlat can be calculated by OGGM later, as we have seen earlier. Here, we prefer to keep the Area computed by the RGI for consistency.

In [ ]:
# We use the RGI as template (this avoids strange IO issues)
template = gl.iloc[[0]].copy()
template['geometry'] = merged['geometry'].values

In [ ]:
# Change CenLon, CenLat
cenlon, cenlat = merged.iloc[0].geometry.centroid.xy
template['CenLon'] = cenlon
template['CenLat'] = cenlat

# We sum up the areas
template['Area'] = gl.Area.sum()
template['Area']

In [Recinos et al., (2019)](https://www.the-cryosphere.net/13/2657/2019/tc-13-2657-2019.html) we wanted to estimate a frontal ablation flux for this new outline and compare it with previous estimates found in the literature for the Sawyer Glacier ([McNabb et al., 2015](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2014JF003276)). 

For this reason we kept the Sawyer glacier attributes to the following variables:
`RGIId, GLIMSId, Name`

The `TermType` should be equal to 1, for Marine-terminating. 

In [ ]:
template['TermType'] = 1
template['Name'] = 'Sawyer Glacier merged with RGI60-01.23664'

Now we can write a new shapefile for this glacier. 
We recommend doing this if you have to make several model runs. You can also integrate this outline to your main RGI shapefile:

In [ ]:
cfg.initialize()
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='rgi-case-2', reset=True)
template.crs = salem.wgs84.srs
template.to_file(os.path.join(cfg.PATHS['working_dir'], 'merged.shp'))

### Run OGGM with this new glacier

For simplicity, we do not compute the intersects in this case: **however, we recommend you do do so (see above). In all cases, do not use the intersects provided automatically with OGGM when using custom inventories, as they are likely to be wrong.**

In [ ]:
# Set-up the run
cfg.PARAMS['border'] = 10

# We don't use intersects here
cfg.PARAMS['use_intersects'] = False

# We prefer OGGM to use the area we computed ourselves
cfg.PARAMS['use_rgi_area'] = True

# Use our merged file
rgidf = gpd.read_file(os.path.join(cfg.PATHS['working_dir'], 'merged.shp'))
gdirs = workflow.init_glacier_directories(rgidf, reset=True, force=True)
gdirs

Here we are not able to use the [Pre-processed directories](https://docs.oggm.org/en/stable/input-data.html#pre-processed-directories) and the respective [Processing levels](https://docs.oggm.org/en/stable/input-data.html#preprodir) that OGGM provides for a easy run set up. We can't use this workflow simply because we have a different beginning than OGGM, we have a different RGI! We just need to type more and run all the model task one by one:

In [ ]:
from oggm.workflow import execute_entity_task

# Calculate the DEMs and the masks
execute_entity_task(tasks.define_glacier_region, gdirs, source = 'SRTM')
execute_entity_task(tasks.glacier_masks, gdirs)

# Calculate the Pre-processing tasks
task_list = [
    tasks.compute_centerlines,
    tasks.initialize_flowlines,
    tasks.catchment_area,
    tasks.catchment_width_geom,
    tasks.catchment_width_correction,
]

for task in task_list:
    execute_entity_task(task, gdirs)

note that we used in `tasks.define_glacier_region` SRTM as DEM source which is not the default one in OGGM. 
The default one is NASADEM, but for that you have to register at [NASA Earthdata](https://urs.earthdata.nasa.gov/) 
more info in [dem_sources.ipynb/register](dem_sources.ipynb#register)

In [ ]:
graphics.plot_googlemap(gdirs, figsize=(6, 6));

In [ ]:
graphics.plot_catchment_width(gdirs, corrected=True);

<a id='case3'></a>
## Case 3: start from a completely independent inventory

Refer to [Case 1](#case1) above, where we show how to start from an RGI template. The main message is: get the attributes right and let OGGM compute the area! 

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)